# SARS-ARENA: Structure-based identification of SARS-derived peptides with potential to induce broad protective immunity

## *Workflow 2* - Peptide-HLA Prediction for Conserved SARS-CoV-2 Peptides

Welcome to the Peptide-HLA Prediction for Conserved SARS-CoV-2 Peptides Workflow. This notebook will allow you to model the conserved peptides selected in Workflow 1 with a HLA of interest. 

This workflow consists of 5 steps: 
    1. Obtain peptide and HLA sequences for prediction;
    2. Filter peptides using a sequence-based affinity prediction tool;
    3. Model HLAs;
    4. Model peptide-HLA complexes with APE-Gen;
    5. Structural scoring functions.

**In order to run a cell, first click on the cell, then press shift-enter. The code inside the cell will then be executed. Note that the content of the cell can be executed as Code or Markdown. Also, inside the cell you may find comments to explain a specific command. These comments are marked with "#"**

**BEFORE STARTING THIS WORKFLOW, MAKE SURE YOU FOLLOWED THE INSTRUCTIONS TO UPDATE THE DOCKER IMAGE TO RUN THE PROGRAM MODELLER. Check DOCUMENTATION.html file, available at https://github.com/KavrakiLab/SARS-Arena.**


### Step 1) Obtain peptide and HLA sequences for prediction

In this first part, you will load the peptides and the HLAs you want to be modeled.

#### 1.1. Necessary imports:
Run this cell to make the necessary imports. This cell should be run only one time, unless you close this session and open it again.

In [ ]:
# System-based imports
import os

# Data processing
import pandas as pd
import shutil
import itertools

#For visualization and interaction purposes
import matplotlib as plt
%matplotlib inline
from ipywidgets import widgets
plt.rcParams['figure.figsize'] = [14, 10]
plt.rcParams['figure.dpi'] = 100

# For utility functions used within the code
from SARS_Arena import *

#### 1.2. Setting a working directory:
Choose an appropriate directory for storing all the files or use the default (*Peptide-HLA_Binding_Prediction_Workflow*).

In [ ]:
dir_of_workflow_2 = "./Peptide-HLA_Binding_Prediction_Workflow_2"
os.makedirs(dir_of_workflow_2, exist_ok=True)
os.chdir(dir_of_workflow_2)

#### 1.3. Loading files:


For this workflow, you will need two files:

**File 1)** The peptide list obtained from Workflow 1 (default name: *peptides.list*)
- In this file, each line should correspond to one peptide;
- This file should be located in folder "Peptide_Extraction_Workflow_1A/B/C", unless you changed the name of the folder. If you changed the file location
- **Please, copy this file to the folder you set as working directory (the folder name is described on the cell above)**

**File 2)** A list of HLAs you want to model these peptides (default name: *hlas.list*)
- In this file, each line should correspond to one HLA. **It is important that the HLA name follows the pattern GENE\*ALLELE GROUP:HLA PROTEIN (e.g. A\*02:01, B\*57:01, C\*11:07)**
- **This file should be located in the same folder as the peptides.list (your working directory folder)**
- You can find an example \emph{hlas.list} in the utils folder

Run the cell bellow to load the peptides. If everything runs well, the number of peptides will be displayed.

In [ ]:
peptides = list(pd.read_csv("peptides.list", header=None)[0])
print("Number of peptides:", len(peptides))

Run the cell bellow to load the HLAs. If everything runs well, the number of HLAs will be displayed.

In [ ]:
hlas = list(pd.read_csv("hlas.list", header=None)[0])
print("Number of hlas:", len(hlas))

In order to model the HLAs, we need the protein sequence of these molecules. We use the IMGT/HLA Database to obtain these sequences.

In [ ]:
hla_sequences = fetch_hla_sequences(hlas)

As there could be a missmatch or missing data, we keep only the HLAs that we found sequences for for the rest of the workflow:

In [ ]:
print("Number of hlas with available sequences from IMGT:", len(hla_sequences))

Here, you can see the HLA sequences obtained from IMGT/HLA Database:

In [ ]:
hla_sequences

### Step 2) Filter peptides using a sequence-based affinity prediction tool
In this part, you will be able to filter the peptides according to specific binding values. These values are acquired using a sequence-based scoring function from [MHCflurry tool](https://openvax.github.io/mhcflurry/).

#### 2.1. MHCFlurry Supported HLAs:

As MHCFlurry does not have support over all human alleles, we need to further filter the input alleles so that only the supported ones pass through MHCFlurry:

In [ ]:
hla_sequences = hla_filtering(hla_sequences)

In [ ]:
hla_sequences

#### 2.2. Running MHCflurry:


First, you will need to prepare the input file that consists of two columns - allele and peptide - running the cell below. For more information, refer to https://openvax.github.io/mhcflurry/

In [ ]:
mucflurry_input = pd.DataFrame(list(itertools.product(list(hla_sequences.keys()),peptides)),
                               columns =['allele', 'peptide'])
mucflurry_input.to_csv('mhcflurry_input.csv', index = False)

You can see the input data format below:

In [ ]:
mucflurry_input

A file called "mhcflurry_input.csv" was created in your current folder. You can now run the peptide-HLA prediction using MHCflurry running the cell below.

In [ ]:
mhcflurry_scoring()

After running the cell above, you will get a file called "predictions.csv". For manual checking, you can open this file and check the values. The binding affinity predictions are given as affinities (Kd) in nM in the *mhcflurry_prediction* column. Lower values indicate stronger binders (as a reference, we normally use a threshold of 500nM). The *mhcflurry_prediction_percentile* column gives the percentile of the affinity prediction among a large number of random peptides tested on that allele (range 0 - 100). Lower is stronger (as a reference, we normally use a value of 2%).

In [ ]:
df_predictions = pd.read_csv("predictions.csv")
df_predictions = df_predictions[['allele', 'peptide', 'mhcflurry_prediction']]

In [ ]:
df_predictions.head(5)

Here, we will apply the cutoff of 500nM before proceeding to the structure-based analysis, to retain only the peptides with higher binding probability.

**PS: After running this cell, you can change the cutoff value through the slider located immediately below this cell**

In [ ]:
binder_cutoff = widgets.Text()
mhcflurry_plot_selection(df_predictions, binder_cutoff)

Let's now check the peptides retrieved from the sequence prediction filtering above:

In [ ]:
Filtered_peptides = df_predictions[df_predictions['mhcflurry_prediction'] < int(binder_cutoff.value)]
Filtered_peptides

### Step 3) Model HLAs

The homology modeling step uses MODELLER, which requires a license key. Follow the [DOCUMENTATION](http://127.0.0.1:8888/view/DOCUMENTATION.html) to update the key, before executing the following cells. You will need to update the key one time only.

A subset of HLAs will be created, based on the alleles that passed the sequence-based prediction threshold

In [ ]:
hla_sequences = {key: hla_sequences[key] for key in Filtered_peptides['allele'].to_list()}

In [ ]:
hla_sequences

In [ ]:
model_hlas_MODELLER(hla_sequences)

### Step 4) Model peptide-HLA complexes with APE-Gen

Now, we will model the three-dimensional structure of these peptides in the context of each HLA using *APE-GEN* tool, a fast method to generate peptide-HLA ensembles. You can find more ingormation about this tool in this [link](https://pubmed.ncbi.nlm.nih.gov/30832312/).

This process can take some time, depending on the number of complexes to be modelled. Run the cell below to check the approximated time needed for modelling these structures.

In [ ]:
print("Approximated time needed for modelling peptides with APE-Gen (8 cores per allele):")
print("2 minutes per complex")
print("{:}".format(Filtered_peptides.shape[0]*2)+" minutes for the total of " +str(Filtered_peptides.shape[0])+ " complexes")

The cell below runs APE-gen:

In [ ]:
best_scoring_confs = model_structures(Filtered_peptides)

The peptide-HLA complexes were modelled and can be found on the folder described at the end of each round of modelling (above). APE-GEN generates several files, but if you are interested only in the lowest energy structure, search for the "*openmm_min_energy_system.pdb*" file.

### Step 5) Structural scoring functions

Here we offer the opportunity to rerun the scoring calculation using Autodock, Vina, Vinardo, or our custom scoring function.

#### 5.1) Score modeled complexes with Autodock (AD4), Vina or Vinardo scoring functions

You can use the standard docking scoring functions (Vina, AD4, Vinardo) and our own custom scoring function to rank the modeled structures and refine your picks for further investigation:

In [ ]:
scoring_results = score_structures(best_scoring_confs)

Here are the results for all four scoring functions:

In [ ]:
scoring_results

You are now free to pick 1 of the 4 scoring functions that you think represents your peptide-HLA pairs and filter them based on this scoring function:

In [ ]:
scoring_function = '3pHLA'
energy_cutoff = widgets.Text()
energy_plot_selection(scoring_results, scoring_function, energy_cutoff)

Results from the above process:

In [ ]:
print("Scoring function chosen: " + scoring_function)
print("Energy threshold chosen: " + energy_cutoff.value)
selected = scoring_results[scoring_results[scoring_function] < float(energy_cutoff.value)]
selected

Run the cells below to store the structures you picked above in the folder you want:

In [ ]:
structures_storage_location = "./workflow2_selected_structures"
os.makedirs(structures_storage_location, exist_ok=True)

In [ ]:
store_best_structures(best_scoring_confs, selected, structures_storage_location)